In [1]:
import pandas as pd, numpy as np
import datetime
import openpyxl

from db_utils import write_to_db  
from db_utils import fetch_data 
from utils import *
from data_manipulator import *
from utilities.connections import *

In [2]:
geo_conversions = fetch_data("SELECT id as geo_region_id , name as city FROM geo_regions",connection ="oms")
geo_conversions.loc[len(geo_conversions.index)] = [0,'India'] 
geo_conversions=geo_conversions.sort_values('geo_region_id').reset_index(drop=True)
geo_conversions.head()


Query run time in seconds : 1.0981616973876953


,geo_region_id,city
0,0,India
1,1,Mumbai
2,2,Delhi NCR
3,3,Bangalore
4,4,Hyderabad


In [3]:
q_reactivation='select * from raw_reactivation_personal where geo_region_id <= 30 and geo_region_id >= 0 '
connection='redshift'
reactivation_t=fetch_data(q_reactivation,connection)

q_retention='select * from raw_retention_personal'
connection='redshift'
retention=fetch_data(q_retention,connection)

q_acquistion = 'select * from raw_acquisition_personal'
connection='redshift'
acquistion=fetch_data(q_acquistion,connection)

q_lead = 'select distinct * from raw_leads_personal'
connection='redshift'
leads=fetch_data(q_lead,connection)

Query run time in seconds : 1.4240350723266602
Query run time in seconds : 0.8164279460906982
Query run time in seconds : 3.5826570987701416
Query run time in seconds : 3.624207019805908


In [4]:
reactivation_t['geo_region_id'].unique()

array([ 7,  4,  1, 13, 10,  2,  3,  5,  6,  8,  9, 11, 12, 14, 15])

In [5]:
reactivation_t.head()

,month,geo_region_id,acc_customers,acc_g30_l60,acc_g60_l90,acc_g90_l120,acc_g120,month.1,geo_region_id.1,reactivated_customers,...,reactivated_customers_m2,reactivated_l2_m2,reactivated_g3_m2,reactivated_g30_l60_m2,reactivated_g60_l90_m2,reactivated_g90_l120_m2,reactivated_g120_m2,orders_m2,orders_l2_m2,orders_g3_m2
0,2021-06-01,7,590,61,47,28,454,2021-06-01,7,30,...,2.0,1.0,1.0,2.0,0.0,0.0,0.0,10.0,1.0,9.0
1,2021-07-01,4,151387,10609,13110,12805,114863,2021-07-01,4,6843,...,472.0,363.0,109.0,235.0,92.0,44.0,101.0,1074.0,476.0,598.0
2,2021-08-01,1,217878,16063,11603,11650,178562,2021-08-01,1,9204,...,1838.0,1263.0,575.0,831.0,458.0,202.0,347.0,4546.0,1680.0,2866.0
3,2021-08-01,13,257,11,2,16,228,2021-08-01,13,1,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,2021-09-01,10,1984,443,316,215,1010,2021-09-01,10,64,...,13.0,5.0,8.0,7.0,4.0,0.0,2.0,43.0,7.0,36.0


In [6]:
reactivation_t.month.unique()

array(['2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01',
       '2021-10-01', '2021-11-01', '2021-12-01', '2022-01-01',
       '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01',
       '2022-07-01', '2022-06-01'], dtype=object)

In [7]:
reactivation_t['month'] = pd.to_datetime(reactivation_t['month']).dt.strftime('%Y-%m-%d')
reactivation=reactivation_t.drop(['month.1','month.2', 'month.3','geo_region_id.1','geo_region_id.2','geo_region_id.3'], axis = 1) 
reactivation=reactivation.drop_duplicates()

retention['month'] = pd.to_datetime(retention['month']).dt.strftime('%Y-%m-%d')
retention=retention.drop_duplicates()

acquistion['month'] = pd.to_datetime(acquistion['month']).dt.strftime('%Y-%m-%d')
acquistion=acquistion.drop_duplicates()

leads['month'] = pd.to_datetime(leads['month']).dt.strftime('%Y-%m-%d')
leads=leads.drop_duplicates()

In [8]:
leads=leads.fillna({'geo_region_id':20.0})

leads=leads.groupby(['month','geo_region_id'])['leads'].sum().reset_index() \
    .assign(attribution='New_Leads').append(leads,ignore_index=False)
leads=movecol(leads, 
             cols_to_move=['month','geo_region_id'], 
             ref_col='attribution',
             place='Before')


In [9]:
total_new_leads=leads.loc[leads['attribution']=='New_Leads']

In [10]:
col_list=['leads']
a=len(total_new_leads)
tot=total_new_leads.head(a)    
unique_geo_regions = [0]+tot['geo_region_id'].unique().tolist()
pan_india_totals= tot.reset_index().groupby(['month'])[col_list].sum()
pan_india_totals['geo_region_id'] = 0
pan_india_totals['attribution'] = 'New_Leads' 
pan_india_totals=pan_india_totals.reset_index()


In [11]:
pan_india_totals.to_csv('pan_india_leads.csv',index=False,header=True)

In [12]:
pan_india_totals_acq=pan_india_totals[['month',	'geo_region_id','attribution','leads']]
total_new_leads.tail()

,leads,month,geo_region_id,attribution
220,2098,2022-07-01,13.0,New_Leads
221,1623,2022-07-01,14.0,New_Leads
222,2046,2022-07-01,15.0,New_Leads
223,72,2022-07-01,16.0,New_Leads
224,9547,2022-07-01,20.0,New_Leads


In [13]:
## Appending the Pan India Leads To Total Leads

total_new_leads=total_new_leads.append(pan_india_totals_acq,ignore_index=True)

In [14]:

#pan_india_leads.to_csv('pan_india_leads.csv',index=False,header=True)
total_new_leads.to_csv('tot_leads.csv',index=False,header=True)


In [15]:
col_list_acq=acquistion.columns.values.tolist()
col_list_acq=[n for n in col_list_acq if (n != 'geo_region_id')& (n != 'month') & (n != 'attribution')]
pan_india_totals_acq=get_pan_india_totals_react(acquistion,col_list_acq)

col_list_ret=retention.columns.values.tolist()
col_list_ret= [n for n in col_list_ret if (n != 'geo_region_id')& (n != 'month')]
pan_india_totals_ret=get_pan_india_totals_react(retention,col_list_ret)

col_list_react=reactivation.columns.values.tolist()
col_list_react= [n for n in col_list_react if (n != 'geo_region_id')& (n != 'month')]
pan_india_totals_react=get_pan_india_totals_react(reactivation,col_list_react)


In [16]:
pan_india_totals_acq = movecol(pan_india_totals_acq, 
             cols_to_move=['geo_region_id'], 
             ref_col='month',
             place='After')

pan_india_totals_ret = movecol(pan_india_totals_ret, 
             cols_to_move=['geo_region_id'], 
             ref_col='month',
             place='After')

pan_india_totals_react = movecol(pan_india_totals_react, 
             cols_to_move=['geo_region_id'], 
             ref_col='month',
             place='After')


acquistion=acquistion.append(pan_india_totals_acq,ignore_index=True)
# acquistion=pd.merge(acquistion,total_new_leads,on=['month','geo_region_id'],how='left')
# acquistion=acquistion.fillna({'attribution':'New_Leads'})

retention=retention.append(pan_india_totals_ret,ignore_index=True)
reactivation=reactivation.append(pan_india_totals_react,ignore_index=True)

In [17]:

#Merge The Acquisition DataFrame to Total Leads Data Frame
acquistion=pd.merge(acquistion,total_new_leads,how='left',on=['month','geo_region_id'])


In [18]:
acquistion=acquistion.drop_duplicates()
acquistion.to_csv('leads.csv',index=False,header=True)

In [19]:
acquistion=get_lag_columns(acquistion)
retention=get_lag_columns_ret(retention)
reactivation=get_lag_columns_react(reactivation)

In [20]:
acquistion.sort_values(by=['geo_region_id','month'], inplace=True)
acquistion=get_acquistion_data(acquistion)
acquistion=acquistion[['month',	'geo_region_id',	'customers',	'customer_less_2',	'customer_greater_3',	'orders',	'orders_less_2',	'orders_greater_3',	'customers_m_1',	'customer_less_2_m_1',	'customer_greater_3_m_1',	'orders_m_1',	'orders_less_2_m_1',	'orders_greater_3_m_1',	'customers_m_2',	'attribution',	'leads','Conversion %','order_per_customer','Order/Customer (>=3)','Zero order cust','Orders/Customer_m1','% Retention','% Customer>=3/Active']]

retention.sort_values(by=['geo_region_id','month'], inplace=True)
retention=get_retention_data(retention)

reactivation.sort_values(by=['geo_region_id','month'], inplace=True)
reactivation=get_reactivation_data(reactivation)


In [21]:
reactivation.columns

Index(['month', 'geo_region_id', 'acc_customers', 'acc_g30_l60', 'acc_g60_l90',
       'acc_g90_l120', 'acc_g120', 'reactivated_customers', 'reactivated_l2',
       'reactivated_g3', 'reactivated_g30_l60', 'reactivated_g60_l90',
       'reactivated_g90_l120', 'reactivated_g120', 'orders', 'orders_l2',
       'orders_g3', 'reactivated_customers_m1', 'reactivated_l2_m1',
       'reactivated_g3_m1', 'reactivated_g30_l60_m1', 'reactivated_g60_l90_m1',
       'reactivated_g90_l120_m1', 'reactivated_g120_m1', 'orders_m1',
       'orders_l2_m1', 'orders_g3_m1', 'reactivated_customers_m2',
       'reactivated_l2_m2', 'reactivated_g3_m2', 'reactivated_g30_l60_m2',
       'reactivated_g60_l90_m2', 'reactivated_g90_l120_m2',
       'reactivated_g120_m2', 'orders_m2', 'orders_l2_m2', 'orders_g3_m2',
       'orders_lag', 'reactivated_customers_lag', 'Orders/Customer',
       '%Customer>=3/Total Reactivated', 'percentage reactivation',
       'zero_order_customer', '% Retention', 'Orders/Customer_m1

In [22]:
## Acquisition

selected_columns = ['geo_region_id','month','Conversion %','leads','customers','customer_less_2','customer_greater_3','orders','orders_less_2',
                    'orders_greater_3','order_per_customer','Order/Customer (>=3)']
first_df_dict_acq=get_data_subset_acq(selected_columns,acquistion)

selected_columns =['geo_region_id','month','customers_m_1','customer_less_2_m_1','customer_greater_3_m_1','customers_m_2','Zero order cust','orders_m_1',
                    'orders_less_2_m_1','orders_greater_3_m_1','Orders/Customer_m1']
second_df_dict_acq=get_data_subset_acq(selected_columns,acquistion)

selected_columns =['geo_region_id','month','% Retention','% Customer>=3/Active']
third_df_dict_acq=get_data_subset_acq(selected_columns,acquistion)

## Reactivation
selected_columns =['geo_region_id','month','acc_customers','acc_g30_l60','acc_g60_l90','acc_g90_l120','acc_g120']
first_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)

selected_columns = ['geo_region_id','month','reactivated_customers','reactivated_l2','reactivated_g3','orders',
                        'orders_l2','orders_g3','Orders/Customer']
second_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)
    
selected_columns = ['geo_region_id','month','percentage reactivation','%Customer>=3/Total Reactivated','% Retention','reactivated_g30_l60','reactivated_g60_l90','reactivated_g90_l120','reactivated_g120']
third_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)
    
selected_columns = ['geo_region_id','month','reactivated_customers_m1','reactivated_l2_m1','reactivated_g3_m1',
                        'reactivated_customers_m2','zero_order_customer']
fourth_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)
    
selected_columns =['geo_region_id','month','orders_m1','orders_l2_m1','orders_g3_m1','reactivated_g30_l60_m1','reactivated_g60_l90_m1',
                       'reactivated_g90_l120_m1']
fifth_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)


## Retention 
selected_columns = ['geo_region_id','month','customers_retained','less_than_two_order_customers','greater_than_two_order_customers',]
first_df_dict = get_data_subset(selected_columns, retention)
    
selected_columns = ['geo_region_id','month','total_orders','orders_less_than_2','orders_greater_than_2','order_per_customer']
second_df_dict = get_data_subset(selected_columns, retention)

selected_columns = ['geo_region_id','month','m1_customers_retained','m1_less_than_two_order_customers','m1_greater_than_two_order_customers',
                    'm1_retention_history','zero_order_customer','M-1_order_per_customer','%_Retention']
third_df_dict = get_data_subset(selected_columns, retention)



In [23]:
[0]+sorted(reactivation_t['geo_region_id'].unique().tolist())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [24]:
df_dict = {}
#for geo_region in [0,1,2,3,4,5,6,8,10]:
for geo_region in [0]+sorted(reactivation_t['geo_region_id'].unique().tolist()):
    df_dict[str(geo_region)] = {
        'Acquisition Customers & Orders':acquisition_first_metric(first_df_dict_acq[geo_region]),
        'Retention for Acquisition Base':acquisition_second_metric(second_df_dict_acq[geo_region]),
        'Retention % for Acquistion Base':acquisition_third_metric(third_df_dict_acq[geo_region]),
        'Retained Customer Base':retention_first_metric(first_df_dict[geo_region]),
        'Retained Customers Orders':retention_second_metric(second_df_dict[geo_region]),
        'M-1 Retention' :retention_last_month_metric(third_df_dict[geo_region]),
        'Accumulated Base':reactivation_first_metric(first_df_dict_reactivation[geo_region]),
        'Reactivated Customers':reactivation_customer_metric(second_df_dict_reactivation[geo_region]),
        'Reactivation Base':reactivation_days_metric(third_df_dict_reactivation[geo_region]),
        'Retention for Reactivation Base':reactivation_second_metric(fourth_df_dict_reactivation[geo_region]),
        'NextMonth Retention for Reactivation Base':reactivation_third_metric(fifth_df_dict_reactivation[geo_region])
        
        
    }        

In [25]:
# for key in df_dict['7'].keys():
#     df_dict['7'][key].fillna(0, inplace=True)

In [26]:
df_dict['11']['Acquisition Customers & Orders']

month,2021-06-01,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01
Conversion %,7.14,4.55,6.93,7.44,10.47,14.44,11.80,9.69,16.01,19.41,25.11,25.26,22.82,17.47
New_Leads,42.00,1452.00,1588.00,2123.00,2769.00,2340.00,3534.00,4632.00,3773.00,3384.00,3106.00,3710.00,4269.00,4940.00
Customer,3.00,66.00,110.00,158.00,290.00,338.00,417.00,449.00,604.00,657.00,780.00,937.00,974.00,863.00
Customers <=2,3.00,65.00,106.00,149.00,271.00,303.00,376.00,440.00,580.00,626.00,753.00,900.00,938.00,828.00
Customers >=3,0.00,1.00,4.00,9.00,19.00,35.00,41.00,9.00,24.00,31.00,27.00,37.00,36.00,35.00
Order,4.00,75.00,124.00,192.00,381.00,501.00,603.00,539.00,740.00,817.00,949.00,1132.00,1173.00,1060.00
Order <=2,4.00,72.00,110.00,162.00,313.00,356.00,430.00,499.00,642.00,692.00,821.00,971.00,1027.00,924.00
Order >=3,0.00,3.00,14.00,30.00,68.00,145.00,173.00,40.00,98.00,125.00,128.00,161.00,146.00,136.00
order_per_customer,1.33,1.14,1.13,1.22,1.31,1.48,1.45,1.20,1.23,1.24,1.22,1.21,1.20,1.23
Order/Customer (>=3),NaN,3.00,3.50,3.33,3.58,4.14,4.22,4.44,4.08,4.03,4.74,4.35,4.06,3.89


In [27]:
for key in df_dict['7'].keys():
    df_dict['7'][key]= df_dict['7'][key][list( filter(lambda x: x>= '2021-06-01', df_dict['7']['Acquisition Customers & Orders'].columns))]

In [28]:
for key in df_dict['11'].keys():
    df_dict['11'][key]= df_dict['11'][key][list( filter(lambda x: x>= '2021-06-01', df_dict['11']['Acquisition Customers & Orders'].columns))]

In [29]:
for key in df_dict['12'].keys():
    df_dict['12'][key]= df_dict['12'][key][list( filter(lambda x: x>= '2021-08-01', df_dict['12']['Acquisition Customers & Orders'].columns))]

for key in df_dict['13'].keys():
    df_dict['13'][key]= df_dict['13'][key][list( filter(lambda x: x>= '2021-09-01', df_dict['13']['Acquisition Customers & Orders'].columns))]

for key in df_dict['14'].keys():
    df_dict['14'][key]= df_dict['14'][key][list( filter(lambda x: x>= '2021-10-01', df_dict['14']['Acquisition Customers & Orders'].columns))]
    
for key in df_dict['15'].keys():
    df_dict['15'][key]= df_dict['15'][key][list( filter(lambda x: x>= '2021-12-01', df_dict['15']['Acquisition Customers & Orders'].columns))]
    

In [30]:
    def generate_excel_workbook():
        """ Creates an excel sheet that has necessary information """
        #geo_regions = [0,1,2,3,4,5,6,8,10]
        geo_regions = [0]+sorted(reactivation_t['geo_region_id'].unique().tolist())
        data_frame_dicts = {}
        for geo_region in geo_regions:
            dataframe_array = []
            for table in df_dict[str(geo_region)]:
                write_to_excel = df_dict[str(geo_region)][table].reset_index()
                write_to_excel.rename(columns = {'index' :table}, inplace = True)
                dataframe_array.append(write_to_excel)
            sheet_name = "geo_region_id_" + str(geo_region)
            data_frame_dicts[sheet_name] = dataframe_array
        run_time = (str(datetime.datetime.now()).split(" ")[1].split(".")[0]).replace(":","_")
        sheet_name = "./demand_report_workbook_personal" + ".xlsx" 
        writer = pd.ExcelWriter(sheet_name,engine ='xlsxwriter') 
        workbook  = writer.book
        bold = workbook.add_format({'bold': True})
#         for key in data_frame_dicts.keys():
#             row = 0
#             worksheet.write(row, col, key)
#             row += 1 


        for geo_region, data in data_frame_dicts.items():
            print(data)
            col = 1
            row = 1
            for df in data :
                print("hello")
                add_col = len(df.columns)
                add_row = len(df.index)
                df.to_excel(writer, sheet_name = str(geo_region), 
                     startrow = row, startcol = col, index = False)
                worksheet = writer.sheets[geo_region]
                header_format = workbook.add_format({
                              'bold': True,
                              'text_wrap': True,
                              'valign': 'top',
                              'fg_color': '#003366',
                              'font_color': 'white',
                              'border': 1,
                              'num_format': 'mmm-yy' })
               
                for f, b in zip(df.columns.values, list(range(col, col + add_col))):
                    worksheet.write(row, b , f, header_format)
                row = row + add_row + 4
        writer.save()


In [31]:
generate_excel_workbook()

[month Acquisition Customers & Orders  2021-06-01  2021-07-01  2021-08-01  \
0                       Conversion %       36.57       37.07       39.41   
1                          New_Leads   139449.00   167742.00   179663.00   
2                           Customer    50995.00    62181.00    70811.00   
3                      Customers <=2    48005.00    58764.00    66924.00   
4                      Customers >=3     2990.00     3417.00     3887.00   
5                              Order    68798.00    82113.00    92475.00   
6                          Order <=2    54675.00    66520.00    74905.00   
7                          Order >=3    14123.00    15593.00    17570.00   
8                 order_per_customer        1.35        1.32        1.31   
9               Order/Customer (>=3)        4.72        4.56        4.52   

month  2021-09-01  2021-10-01  2021-11-01  2021-12-01  2022-01-01  2022-02-01  \
0           36.53       36.13       37.72       37.33       35.49       39.39   


5           653.0       904.0       839.0      1013.0      1043.0       969.0  ]
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
[month Acquisition Customers & Orders  2021-06-01  2021-07-01  2021-08-01  \
0                       Conversion %       38.31       39.39       43.37   
1                          New_Leads    20747.00    24989.00    28295.00   
2                           Customer     7948.00     9844.00    12271.00   
3                      Customers <=2     7626.00     9403.00    11744.00   
4                      Customers >=3      322.00      441.00      527.00   
5                              Order     9816.00    12226.00    15254.00   
6                          Order <=2     8503.00    10471.00    13034.00   
7                          Order >=3     1313.00     1755.00     2220.00   
8                 order_per_customer        1.24        1.24        1.24   
9               Order/Customer (>=3)        4.08        3.98        4.21   

month  2021-09-

5            42.0        58.0        61.0        80.0        87.0        67.0  ]
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
[month Acquisition Customers & Orders  2021-06-01  2021-07-01  2021-08-01  \
0                       Conversion %       14.40       13.22       13.81   
1                          New_Leads      896.00     1838.00     1998.00   
2                           Customer      129.00      243.00      276.00   
3                      Customers <=2      121.00      227.00      253.00   
4                      Customers >=3        8.00       16.00       23.00   
5                              Order      157.00      318.00      427.00   
6                          Order <=2      129.00      250.00      284.00   
7                          Order >=3       28.00       68.00      143.00   
8                 order_per_customer        1.22        1.31        1.55   
9               Order/Customer (>=3)        3.50        4.25        6.22   

month  2021-09-

5             2.0         1.0         2.0         4.0         3.0         6.0  ]
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
[month Acquisition Customers & Orders  2021-06-01  2021-07-01  2021-08-01  \
0                       Conversion %       23.03       26.73       27.21   
1                          New_Leads     1242.00     1515.00     1694.00   
2                           Customer      286.00      405.00      461.00   
3                      Customers <=2      272.00      385.00      436.00   
4                      Customers >=3       14.00       20.00       25.00   
5                              Order      369.00      546.00      598.00   
6                          Order <=2      306.00      439.00      494.00   
7                          Order >=3       63.00      107.00      104.00   
8                 order_per_customer        1.29        1.35        1.30   
9               Order/Customer (>=3)        4.50        5.35        4.16   

month  2021-09-

5             3.0         8.0         4.0         5.0  ]
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
[month Acquisition Customers & Orders  2021-09-01  2021-10-01  2021-11-01  \
0                       Conversion %        2.22        5.34        8.04   
1                          New_Leads       90.00     1142.00     1107.00   
2                           Customer        2.00       61.00       89.00   
3                      Customers <=2        2.00       55.00       82.00   
4                      Customers >=3        0.00        6.00        7.00   
5                              Order        2.00       85.00      122.00   
6                          Order <=2        2.00       65.00       92.00   
7                          Order >=3        0.00       20.00       30.00   
8                 order_per_customer        1.00        1.39        1.37   
9               Order/Customer (>=3)         NaN        3.33        4.29   

month  2021-12-01  2022-01-01  2022-02-

## Share the Below Generated Excel Report  - demand_report_workbook.xlsx

In [32]:
### Extra Totals | Formatting & Designing the Sheet(s) & Workbook | Styling the Workbook 
## Formatting & Designing the Workbook

from openpyxl import load_workbook
from openpyxl import Workbook
wb = openpyxl.load_workbook('demand_report_workbook_personal.xlsx')
sheet=wb.active
sheets=wb.sheetnames[0:len(wb.sheetnames)]
for sheet in sheets:    
    ##Fixing the Width of the Metric Header Column
    wb[sheet].column_dimensions['B'].width = 36    
    ## Freezing the Window Pane
    wb[sheet].freeze_panes = 'C2'    
    ## Removing the Grid Lines
    wb[sheet].sheet_view.showGridLines = False
    
wb.save('demand_report_workbook_personal.xlsx')     

## Getting the Grand Total of Orders for Pan India & All City

from openpyxl.formula.translate import Translator
wb = openpyxl.load_workbook('demand_report_workbook_personal.xlsx')
sheet=wb.active
i=sheet.max_row+4
sheets=wb.sheetnames[0:len(wb.sheetnames)]
#sheets=[n for n in sheets if (n != 'Pune')]
for sheet in sheets:
    print(sheet)
    for j in range(3,wb[sheet].max_column+1):
        ws=wb[sheet]
        wb[sheet].cell(row=i,column=2).value='Total Customers'
        ws.cell(row=i,column=j).value=(ws.cell(row=5,column=j).value)+(ws.cell(row=36,column=j).value)+(ws.cell(row=71,column=j).value)
        wb[sheet].cell(row=i+1,column=2).value='Total Orders'
        wb[sheet].cell(row=i+1,column=j).value=(wb[sheet].cell(row=8,column=j).value)+(wb[sheet].cell(row=43,column=j).value)+(wb[sheet].cell(row=74,column=j).value)  
                
wb.save("demand_report_workbook_personal.xlsx")  

## Workbook styling
## Border Styling & Beautification the Grand Totals

from openpyxl.styles.borders import Border, Side
from openpyxl import Workbook

dash_border = Border(left=Side(style='mediumDashDot'), 
                     right=Side(style='mediumDashDot'), 
                     top=Side(style='mediumDashDot'), 
                     bottom=Side(style='mediumDashDot'))

from openpyxl.formula.translate import Translator
wb = openpyxl.load_workbook('demand_report_workbook_personal.xlsx')
sheet=wb.active
sheets=wb.sheetnames[0:len(wb.sheetnames)]
sheets
for sheet in sheets:
    wb[sheet].cell(row=111,column=2).border = dash_border
    wb[sheet].cell(row=112,column=2).border = dash_border        
    
wb.save("demand_report_workbook_personal.xlsx")  

## Coloring & Naming the Sheets to City & Individual Tab 
## Naming the Sheets
from openpyxl import load_workbook
from openpyxl import Workbook
wb=openpyxl.load_workbook('demand_report_workbook_personal.xlsx') 

for i in [0]+sorted(reactivation_t['geo_region_id'].unique().tolist()):
    try:
        indx = (geo_conversions[geo_conversions['geo_region_id']==i]).index[0]
        wb['geo_region_id_{a}'.format(a=i)].title = geo_conversions.iloc[indx]['city']
    except Exception:
        pass    


# ws1=wb['geo_region_id_0'].title='India'
# ws2=wb['geo_region_id_1'].title='Mumbai'
# ws3=wb['geo_region_id_2'].title='Delhi'
# ws4=wb['geo_region_id_3'].title='Bangalore'
# ws5=wb['geo_region_id_4'].title='Hyderabad'
# ws6=wb['geo_region_id_5'].title='Chennai'
# ws7=wb['geo_region_id_6'].title='Ahmedabad'
# ws8=wb['geo_region_id_8'].title='Pune'
# ws9=wb['geo_region_id_10'].title='Surat'

wb.save('demand_report_workbook_personal.xlsx') 


## India Flag Pattern
color_map={0:'00FF6600',1:'00FFFFFF',2:'0000FF00'}
wb = openpyxl.load_workbook('demand_report_workbook_personal.xlsx')
# for sheet in wb.worksheets:

for index,name in enumerate(wb.sheetnames):
    color = color_map[index%3]
    wb[name].sheet_properties.tabColor = color



#     wb['India'].sheet_properties.tabColor = "00FF6600"
#     wb['Mumbai'].sheet_properties.tabColor = "00FFFFFF"
#     wb['Delhi'].sheet_properties.tabColor = "0000FF00"
#     wb['Bangalore'].sheet_properties.tabColor = "00FF6600"
#     wb['Chennai'].sheet_properties.tabColor = "00FFFFFF"
#     wb['Hyderabad'].sheet_properties.tabColor = "0000FF00"
#     wb['Ahmedabad'].sheet_properties.tabColor = "00FF6600"
#     wb['Pune'].sheet_properties.tabColor = "00FFFFFF"
#     wb['Surat'].sheet_properties.tabColor = "0000FF00"

wb.save("demand_report_workbook_personal.xlsx")




geo_region_id_0
geo_region_id_1
geo_region_id_2
geo_region_id_3
geo_region_id_4
geo_region_id_5
geo_region_id_6
geo_region_id_7
geo_region_id_8
geo_region_id_9
geo_region_id_10
geo_region_id_11
geo_region_id_12
geo_region_id_13
geo_region_id_14
geo_region_id_15


In [33]:
sheets

['geo_region_id_0',
 'geo_region_id_1',
 'geo_region_id_2',
 'geo_region_id_3',
 'geo_region_id_4',
 'geo_region_id_5',
 'geo_region_id_6',
 'geo_region_id_7',
 'geo_region_id_8',
 'geo_region_id_9',
 'geo_region_id_10',
 'geo_region_id_11',
 'geo_region_id_12',
 'geo_region_id_13',
 'geo_region_id_14',
 'geo_region_id_15']